# Reshape Author Affs and finalize base dataset

Reshaping the Author and affiliations data

In [ ]:
import pandas as pd
import numpy as np
import json
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)


In [3]:
base_path="/Users/sijiawu/Work/Thesis/Data"
alt_input_base_path=base_path+"/021_affiliation_restructure"
subinput_base_path=base_path+"/022_author_combine_w_affils"
input_base_path=base_path+"/023_author_recon_w_affils"
output_base_path= base_path+"/024_author_consolidation_summary"

In [4]:
content_ex=['MISC',  'Discussion', 'Review', 'Review2',"Errata"]
content=['Article', 'Comment', 'Reply', 'Rejoinder']
jid=["aer","ecta","jpe","qje","res"]

In [ ]:
with open(input_base_path+"/author_proc_all_w_red.json") as f: 
    data = f.read() 
proc_auths_all = json.loads(data) 

proc_auths=pd.read_pickle(input_base_path+"/auth_proc_w_aff_flat.pkl")
j_data=pd.read_pickle(subinput_base_path+"/022_merged_proc_scopus_inception_with_auth_split_2020_content_type_corrected.pkl")

In [158]:
# Failure cases
proc_auths_all['https://www.jstor.org/stable/23469808'] # wendy a. stock case. just no affiliations recorded for whole article
proc_auths_all['https://www.jstor.org/stable/42920725'] # no recorded affiliation for 1 of 2 authors
proc_auths_all["https://www.jstor.org/stable/42920859"] # aff_main_final of atleast one entry is "none"
proc_auths_all["https://www.jstor.org/stable/40928972"] # null author case

{'authors': {}, 'year': 2010, 'content_type': 'Article', 'jid': 'ecta'}

In [ ]:
# flattening auth aff dictionary to usable set

aff_out=[]
issue=[]
for i in proc_auths_all.keys():
    if proc_auths_all[i]['content_type'] not in content:
        continue
    if proc_auths_all[i]['year']<1940:
        continue
    temp={
        "url":i,
        'year': proc_auths_all[i]["year"],
        'content_type': proc_auths_all[i]["content_type"],
        'jid': proc_auths_all[i]["jid"],
        }
    for j in proc_auths_all[i]["authors"].keys(): # result is only authors who are not nan make it in since nan authors will have an empty author dict so no [] returns
        # print(i)
        # print(proc_auths_all[i]["authors"][j]["resolution"])
        temp2={"auth_num": j, 'raw':proc_auths_all[i]["authors"][j]["raw"],"resolution":proc_auths_all[i]["authors"][j]["resolution"]}
        for k in proc_auths_all[i]["authors"][j]['affs'].keys():
            try:
                temp3={"aff_num":k, 'affiliation': proc_auths_all[i]["authors"][j]['affs'][k]['aff_main_final']}
                aff_out.append(temp|temp2|temp3)
            except:
                print(i)
                print('"'+proc_auths_all[i]["authors"][j]['affs'][k]['aff_main_final']+'"')
                issue.append(i)
                

In [ ]:

# proc_auths creation ctd
aff_all=pd.DataFrame(aff_out)
aff_all["id"]=aff_all["url"].str.split("/").str[-1]

# aff list unique
affs_uni_=aff_all['affiliation'].unique()
affs_uni_.sort()
len(affs_uni_)

2392

In [124]:
proc_auths["id"]=proc_auths["url"].str.split("/").str[-1]

print(proc_auths["url"].unique().shape)
print(proc_auths.shape)
display(proc_auths["content_type"].value_counts())
display(proc_auths.head())


(29382,)
(46425, 27)


content_type
Article      43420
Comment       1807
Reply         1064
Rejoinder      134
Name: count, dtype: int64

,auth_ord,raw,init,suffix,a1,a2,a3,last,affs,year,...,fl,a1_tk_count,ini,inil,r_a1,r_a1_name,p_a1,p_a1_name,resolution,id
0,0,"Enke, Benjamin",benjamin enke,[],benjamin enke,b. enke,b. enke,enke,"{harvard university, national bureau of economic research - nber}",2020,...,benjamin enke,2,b,b. enke,1521,benjamin enke,NaN,,benjamin enke,qjaa012
1,0,Lauren Falcao Bergquist,lauren falcao bergquist,[],lauren f. bergquist,l. f. bergquist,l. bergquist,bergquist,{university of michigan},2020,...,lauren bergquist,3,l,l. bergquist,6361,lauren f. bergquist,NaN,,lauren f. bergquist,26966478
2,1,Michael Dinerstein,michael dinerstein,[],michael dinerstein,m. dinerstein,m. dinerstein,dinerstein,{university of chicago},2020,...,michael dinerstein,2,m,m. dinerstein,3471,michael dinerstein,NaN,,michael dinerstein,26966478
3,0,Dominic Coey,dominic coey,[],dominic coey,d. coey,d. coey,coey,{facebook},2020,...,dominic coey,2,d,d. coey,5380,dominic coey,NaN,,dominic coey,26966479
4,1,Bradley J. Larsen,bradley j. larsen,[],bradley j. larsen,b. j. larsen,b. larsen,larsen,"{stanford university, national bureau of economic research - nber}",2020,...,bradley larsen,3,b,b. larsen,11847,bradley j. larsen,NaN,bradley j. larsen,bradley j. larsen,26966479


In [125]:
res=pd.DataFrame(proc_auths["resolution"].unique()).rename(columns={0: "resolution"})
res=res.sort_values(by="resolution").reset_index(drop=True).reset_index().rename(columns={"index":"id_author"})

res.to_excel(output_base_path+"/024_consolidated_author_name_list.xlsx", index=False)

In [126]:
proc_auths_auid=pd.merge(proc_auths,res,how="left", on="resolution")
proc_auths_exp=pd.merge(aff_all,res,how="left", on="resolution")

In [127]:
proc_auths_exp.to_excel(output_base_path+"/024_auth_aff_expanded.xlsx", index=False)
proc_auths_exp.to_pickle(output_base_path+"/024_auth_aff_expanded.pkl")

In [128]:
sub_arr=['jid', 
        'url',
        'id',
        'year',
        'content_type',
        'auth_ord', 
        'raw', 
        'init',
        'init_order', 
        'suffix', 
        'a1',
        'a1_order', 
        'a2',
        'a2_order', 
        'a3', 
        'a3_order',
        'last', 
        'fl', 
        'ini', 
        'inil', 
        'affs', 
        'resolution', 
        'id_author'
        ]

proc_auths_auid[sub_arr].to_excel(output_base_path+"/024_auth_proc_w_aff_flat_sub_auid.xlsx", index=False)
proc_auths_auid[sub_arr].to_pickle(output_base_path+"/024_auth_proc_w_aff_flat_sub_auid.pkl")

# Affiliations are in a set

In [129]:
j_data.to_excel(output_base_path+"/024_merged_proc_scopus_inception_with_auth_split_2020_content_type_corrected.xlsx", index=False)
j_data.to_pickle(output_base_path+"/024_merged_proc_scopus_inception_with_auth_split_2020_content_type_corrected.pkl")

In [130]:
proc_auths_auid.head()

,auth_ord,raw,init,suffix,a1,a2,a3,last,affs,year,...,a1_tk_count,ini,inil,r_a1,r_a1_name,p_a1,p_a1_name,resolution,id,id_author
0,0,"Enke, Benjamin",benjamin enke,[],benjamin enke,b. enke,b. enke,enke,"{harvard university, national bureau of economic research - nber}",2020,...,2,b,b. enke,1521,benjamin enke,NaN,,benjamin enke,qjaa012,1406
1,0,Lauren Falcao Bergquist,lauren falcao bergquist,[],lauren f. bergquist,l. f. bergquist,l. bergquist,bergquist,{university of michigan},2020,...,3,l,l. bergquist,6361,lauren f. bergquist,NaN,,lauren f. bergquist,26966478,8632
2,1,Michael Dinerstein,michael dinerstein,[],michael dinerstein,m. dinerstein,m. dinerstein,dinerstein,{university of chicago},2020,...,2,m,m. dinerstein,3471,michael dinerstein,NaN,,michael dinerstein,26966478,9994
3,0,Dominic Coey,dominic coey,[],dominic coey,d. coey,d. coey,coey,{facebook},2020,...,2,d,d. coey,5380,dominic coey,NaN,,dominic coey,26966479,3372
4,1,Bradley J. Larsen,bradley j. larsen,[],bradley j. larsen,b. j. larsen,b. larsen,larsen,"{stanford university, national bureau of economic research - nber}",2020,...,3,b,b. larsen,11847,bradley j. larsen,NaN,bradley j. larsen,bradley j. larsen,26966479,1595


In [131]:
res.head()

,id_author,resolution
0,0,a. a. brewer
1,1,a. a. payne
2,2,a. a. ryvkin
3,3,a. a. sampson
4,4,a. a. schmid


In [132]:
proc_auths_exp.head()

,url,year,content_type,jid,auth_num,raw,resolution,aff_num,affiliation,id,id_author
0,https://doi.org/10.1093/qje/qjaa012,2020,Article,qje,0,"Enke, Benjamin",benjamin enke,1,harvard university,qjaa012,1406
1,https://doi.org/10.1093/qje/qjaa012,2020,Article,qje,0,"Enke, Benjamin",benjamin enke,2,national bureau of economic research - nber,qjaa012,1406
2,https://www.jstor.org/stable/26966478,2020,Article,aer,0,Lauren Falcao Bergquist,lauren f. bergquist,1,university of michigan,26966478,8632
3,https://www.jstor.org/stable/26966478,2020,Article,aer,1,Michael Dinerstein,michael dinerstein,1,university of chicago,26966478,9994
4,https://www.jstor.org/stable/26966479,2020,Article,aer,0,Dominic Coey,dominic coey,1,facebook,26966479,3372


In [133]:
proc_auths_auid["id"].unique().shape

(29382,)

# Skeletons to fix
To completely resolve, 4 example failure cases must pass. For the rest of the code sections, I show the problems with the data so far. They are issues to resolve but do not materially effect the outcome down the line. I state these here as known issues, and their impact is neglible because there's so few of them compared to the dataset. 

- Nan values in author names: the assumption should be that they are not articles, I have made an exception in the case of ecta because there seems to be a section of just abstracts for two straight issues. They should not be considered articles. Oversight that can be easily fixed.
- Consolidating the author affiliations input files into one single file. Having MJ's stuff as default and restructuring it is messy. Make it all into Mturk format and treat as raw file. Should actually go through this anyways.
- Nan values for affiliations. The rule should be all authors, article pairings should have an entry in processed authors output where if for that author, article pairing they happen to have no affilitation then their affiliation is "none". See case of Wendy A. Stock for id 23469808. There exists an empty entry in proc_auths_auid because it was added back in as nulls but this paper author set actually has an affiliation it was just missed. See above list.
- JPE has these end of issue suggestions. They are at best reviews and should be removed. It's not identifiable via bibtex citations but it is marked on JSTOR article landings as a "suggested by ...". Easiest way so far: sort by page length. Probably easiest way to sus out errata, corrections and corrigenda as well which are usually no affiliation entries or nan author entries.
- In the expanded dataframe determined by, (author_id, paper_id, affiliation) combo - if there is atleast one affiliation for that author paper pairing, then a secondary affiliation should not be "none". In fact, that "none" affiliation row should be dropped.
- The non-expanded dataframes proc_auths and proc_auths_auid has a single row per author, paper_id combo and affiliations are in a set {} object.
- There are cases where the author just does not have an affiliation in which case they would disappear from the expanded dataframe. It really might not be so ideal to store this as a relational database even if the data is easier to work with. I can't get an actual calc of total author with the expanded dataframe alone if they just disappear. Alt: empty affiliation author paper pairings should just default to none which we can flag and make an adjustment of it in calculations.
- merging scopus data into jstor dataframe would be great if it was one-to-one. But turns out it is not. There is one-to-many and many-to-one which means each time you take counts etc of just jstor you have to run a unique function on URL or scopus id if looking at scopus alone. In true rdbms it's better to have a linking table that is just pairs of jstor_id, scopus_id. This is extremely annoying because of how many joins you'd need to construct graphs etc. so it may be just something to take note of rather than actually adjusting the db structure.


For now, I am going to leave it. We basically can't use proc_auths_exp because it loses too much info when the author has no affiliations.



In [135]:
j_data.columns

Index(['issue_url', 'author', 'title', 'journal', 'volume', 'number', 'pages',
       'year', 'ISSN', 'abstract', 'URL', 'publisher', 'content_type', 'type',
       'jid', 'author_split', 'urldate', 'reviewed-author', 'uploaded',
       'title_10', 'URL_og', 'number_og', 'title_og', 'author_og', 'pages_og',
       'j_fix', 'scopus_jid', 'scopus_id', 'scopus_authorgroup',
       'scopus_authors', 'scopus_affiliations', 'scopus_references',
       'scopus_author_full_names', 'scopus_title', 'scopus_year',
       'scopus_source_title', 'scopus_volume', 'scopus_issue', 'scopus_art_no',
       'scopus_page_start', 'scopus_page_end', 'scopus_page_count',
       'scopus_cited_by', 'scopus_doi', 'scopus_abstract', 'scopus_publisher',
       'scopus_document_type', 'scopus_publication_stage',
       'scopus_open_access', 'scopus_source', 'scopus_eid', 'scopus_title_og',
       'scopus_volume_og', 'scopus_issue_og', 'scopus_page_start_og',
       'scopus_page_end_og', 'scopus_year_og', 's_fix', 

In [ ]:
proc_auths_all['https://www.jstor.org/stable/23469808'] # wendy a. stock case. just no affiliations recorded for whole article
proc_auths_all['https://www.jstor.org/stable/42920725'] # no recorded affiliation for 1 of 2 authors
proc_auths_all["https://www.jstor.org/stable/42920859"] # aff_main_final of atleast one entry is "none"
proc_auths_all["https://www.jstor.org/stable/40928972"] # null author case

In [ ]:
# checking the processed auth, aff dictionary

ic=[]
nones=[]

for i in proc_auths_all.keys():
    if proc_auths_all[i]['content_type'] not in content:
        continue
    if proc_auths_all[i]['year']<1940:
        continue

    for j in proc_auths_all[i]["authors"].keys():
    # print(i)
    # print(proc_auths_all[i]["authors"][j]["resolution"])
        if len(proc_auths_all[i]["authors"][j]['affs'])==0:
            ic.append(i)
        else:

            for k in proc_auths_all[i]["authors"][j]['affs'].keys():
                if proc_auths_all[i]["authors"][j]['affs'][k]['aff_main_final']=="none":
                    nones.append(i)


check=[] # checking if all in content URLs from jstor are in the expanded auth, aff df
nan_author=[] # checking for rows with null authors in the expanded auth, aff df
for i in j_data.index:
    if (j_data.loc[i,"year"]>=1940) & (j_data.loc[i,"content_type"] in content):
        try:
            auth_count=len(j_data.loc[i,"author_split"])
            # print(j_data.loc[i,"URL"])
            if j_data.loc[i,"URL"] not in proc_auths_exp["url"].values:
                check.append(j_data.loc[i,"URL"])
        except:
            # print(j_data.loc[i,"author_split"])
            nan_author.append(j_data.loc[i,"URL"])  
            if j_data.loc[i,"URL"] in proc_auths_exp["url"].values:
                print("null author entry")     # checks if null author urls made it into the expanded df





In [ ]:
print(len(set(ic))) # number of articles with atleast one author who has no recorded affiliations, a {} entry
print(len(ic)) # number of authors, paper entries affected

In [ ]:
print(len(set(nones))) # number of articles with at least one author who has "none" entry in final main affiliation
print(len(nones)) # number of authors, paper entries affected

In [ ]:
print(len(set(check))) # checking if all article URLs from jstor are in the expanded auth, aff df
print(len(check)) 

119

In [ ]:
print(len(set(nan_author))) # checking how many nan authors we picked up in j_data for which content_type is set as Article, Rejoinder, Comment, Reply, 3 duplicate likely from scopus duplication. Must set j_data to take unique set or set up linking scopus_jstor df.
print(len(nan_author))
nan_author[0]

458
455


'https://www.jstor.org/stable/40928972'

In [153]:
j_data[j_data["URL"].isin(check)][["title","year","journal","author", "id","URL"]].head()

,title,year,journal,author,id,URL
1752,one essay on dissertation formats in economics,2013,The American Economic Review,Wendy A. Stock and John J. Siegfried,23469808,https://www.jstor.org/stable/23469808
2605,"corrigendum: language, meaning, and games: a model of communication, coordination, and evolution",2009,The American Economic Review,Stefano Demichelis and Jorgen W. Weibull,25592561,https://www.jstor.org/stable/25592561
8322,short-run price effects of the corporate income tax and implications for international trade: comment,1982,The American Economic Review,Albert A. Schulman,1831562,https://www.jstor.org/stable/1831562
8534,the general nonneutrality of income and consumption taxes: reply,1981,The American Economic Review,John Whalley,1806201,https://www.jstor.org/stable/1806201
10184,the rationale of the mean-standard deviation analysis: comment,1974,The American Economic Review,Karl Borch,1808893,https://www.jstor.org/stable/1808893


In [ ]:
j_data[j_data["author_split"].isna() & (j_data["content_type"].isin(content))& (j_data["year"]>=1940)].head() # all null authors in j_data

,issue_url,author,title,journal,volume,number,pages,year,ISSN,abstract,...,scopus_page_start_og,scopus_page_end_og,scopus_year_og,s_fix,scopus_pages,scopus_indicator,id,authors_lower,page_count,author_count
28460,https://www.jstor.org/stable/10.2307/i40041779,NaN,corrigendum to 'uniform inference in autoregressive models',Econometrica,78,5,1773-1773,2010,"00129682, 14680262",NaN,...,1773,NaN,2010.0,0.0,1773,2,40928972,NaN,NaN,0
29351,https://www.jstor.org/stable/10.2307/i353369,NaN,correction: cointegration and dynamic simultaneous equations model,Econometrica,68,5,1293-1293,2000,"00129682, 14680262",NaN,...,NaN,NaN,NaN,NaN,NaN,0,2999451,NaN,NaN,0
30462,https://www.jstor.org/stable/10.2307/i332686,NaN,a demand system rank theorem,Econometrica,57,3,701-705,1989,"00129682, 14680262",NaN,...,NaN,NaN,NaN,NaN,NaN,0,1911061,NaN,NaN,0
31032,https://www.jstor.org/stable/10.2307/i332701,NaN,"alfred cowles, 3rd, 1891-1984",Econometrica,53,3,683-686,1985,"00129682, 14680262",NaN,...,NaN,NaN,NaN,NaN,NaN,0,1911664,NaN,NaN,0
31155,https://www.jstor.org/stable/10.2307/i332676,NaN,gerhard tintner 1907-1983,Econometrica,52,4,1077-1078,1984,"00129682, 14680262",NaN,...,NaN,NaN,NaN,NaN,NaN,0,1911204,NaN,NaN,0


In [85]:
proc_auths_auid[proc_auths_auid["init"]=="wendy a. stock"]

,auth_ord,raw,init,suffix,a1,a2,a3,last,affs,year,...,a1_tk_count,ini,inil,r_a1,r_a1_name,p_a1,p_a1_name,resolution,id,id_author
1383,0,Wendy A. Stock,wendy a. stock,[],wendy a. stock,w. a. stock,w. stock,stock,{montana state university},2017,...,3,w,w. stock,11744,wendy a. stock,NaN,,wendy a. stock,44250476,14954
3666,0,Wendy A. Stock,wendy a. stock,[],wendy a. stock,w. a. stock,w. stock,stock,{},2013,...,3,w,w. stock,11744,wendy a. stock,NaN,,wendy a. stock,23469808,14954
4590,0,Wendy A. Stock,wendy a. stock,[],wendy a. stock,w. a. stock,w. stock,stock,{montana state university},2011,...,3,w,w. stock,11744,wendy a. stock,NaN,,wendy a. stock,29783737,14954
5751,0,Wendy A. Stock,wendy a. stock,[],wendy a. stock,w. a. stock,w. stock,stock,{montana state university},2009,...,3,w,w. stock,11744,wendy a. stock,NaN,,wendy a. stock,25592468,14954
6886,1,Wendy A. Stock,wendy a. stock,[],wendy a. stock,w. a. stock,w. stock,stock,{montana state university},2006,...,3,w,w. stock,11744,wendy a. stock,NaN,,wendy a. stock,30034690,14954
6888,0,Wendy A. Stock,wendy a. stock,[],wendy a. stock,w. a. stock,w. stock,stock,{montana state university},2006,...,3,w,w. stock,11744,wendy a. stock,NaN,,wendy a. stock,30034691,14954
6891,0,Wendy A. Stock,wendy a. stock,[],wendy a. stock,w. a. stock,w. stock,stock,{montana state university},2006,...,3,w,w. stock,11744,wendy a. stock,NaN,,wendy a. stock,30034692,14954
7520,0,Wendy A. Stock,wendy a. stock,[],wendy a. stock,w. a. stock,w. stock,stock,{montana state university},2004,...,3,w,w. stock,11744,wendy a. stock,NaN,,wendy a. stock,3592894,14954
7523,1,Wendy A. Stock,wendy a. stock,[],wendy a. stock,w. a. stock,w. stock,stock,{montana state university},2004,...,3,w,w. stock,11744,wendy a. stock,NaN,,wendy a. stock,3592895,14954
30594,1,Wendy A. Stock,wendy a. stock,[],wendy a. stock,w. a. stock,w. stock,stock,{kansas state university},1999,...,3,w,w. stock,11744,wendy a. stock,NaN,,wendy a. stock,250091,14954


In [106]:
len(j_data[j_data["year"]>=1940]['issue_url'].unique())

2048

In [ ]:
proc_auths[proc_auths['url']=="https://www.jstor.org/stable/23469808"] # the flattened version of the proc_auths_all dictionary used during consolidation

,auth_ord,raw,init,suffix,a1,a2,a3,last,affs,year,...,fl,a1_tk_count,ini,inil,r_a1,r_a1_name,p_a1,p_a1_name,resolution,id
3666,0,Wendy A. Stock,wendy a. stock,[],wendy a. stock,w. a. stock,w. stock,stock,{},2013,...,wendy stock,3,w,w. stock,11744,wendy a. stock,NaN,,wendy a. stock,23469808
3667,1,John J. Siegfried,john j. siegfried,[],john j. siegfried,j. j. siegfried,j. siegfried,siegfried,{},2013,...,john siegfried,3,j,j. siegfried,6088,john j. siegfried,NaN,,john j. siegfried,23469808


In [146]:
proc_auths[proc_auths["url"]=="https://www.jstor.org/stable/40928972"] # null auth case

,auth_ord,raw,init,suffix,a1,a2,a3,last,affs,year,...,fl,a1_tk_count,ini,inil,r_a1,r_a1_name,p_a1,p_a1_name,resolution,id


In [ ]:
proc_auths_auid[proc_auths_auid["url"]=="https://www.jstor.org/stable/23469808"] # same as above but with an consolidated author id added

,auth_ord,raw,init,suffix,a1,a2,a3,last,affs,year,...,a1_tk_count,ini,inil,r_a1,r_a1_name,p_a1,p_a1_name,resolution,id,id_author
3666,0,Wendy A. Stock,wendy a. stock,[],wendy a. stock,w. a. stock,w. stock,stock,{},2013,...,3,w,w. stock,11744,wendy a. stock,NaN,,wendy a. stock,23469808,14954
3667,1,John J. Siegfried,john j. siegfried,[],john j. siegfried,j. j. siegfried,j. siegfried,siegfried,{},2013,...,3,j,j. siegfried,6088,john j. siegfried,NaN,,john j. siegfried,23469808,7356


In [ ]:
proc_auths_auid[proc_auths_auid["url"]=="https://www.jstor.org/stable/40928972"] # null auth case

,auth_ord,raw,init,suffix,a1,a2,a3,last,affs,year,...,a1_tk_count,ini,inil,r_a1,r_a1_name,p_a1,p_a1_name,resolution,id,id_author


In [ ]:
proc_auths_all["https://www.jstor.org/stable/23469808"] # empty affiliations fields for legit article


{'authors': {'0': {'raw': 'Wendy A. Stock',
   'init': 'wendy a. stock',
   'auth_suffix': [],
   'a1': 'wendy a. stock',
   'a2': 'w. a. stock',
   'a3': 'w. stock',
   'affs': {},
   'affs_min': [],
   'resolution': 'wendy a. stock'},
  '1': {'raw': 'John J. Siegfried',
   'init': 'john j. siegfried',
   'auth_suffix': [],
   'a1': 'john j. siegfried',
   'a2': 'j. j. siegfried',
   'a3': 'j. siegfried',
   'affs': {},
   'affs_min': [],
   'resolution': 'john j. siegfried'}},
 'year': 2013,
 'content_type': 'Article',
 'jid': 'aer'}

In [ ]:
proc_auths_all["https://www.jstor.org/stable/40928972"] # no author article case. aside from whether it's an article or not doesn't really matter

{'authors': {}, 'year': 2010, 'content_type': 'Article', 'jid': 'ecta'}

In [ ]:
proc_auths_exp[proc_auths_exp["url"]=="https://www.jstor.org/stable/23469808"] # proc_auths_exp is the auid version expanded where each affiliation, author, paper combo has a row

,url,year,content_type,jid,auth_num,raw,resolution,aff_num,affiliation,id,id_author


In [ ]:
proc_auths_exp[proc_auths_exp["url"]=="https://www.jstor.org/stable/40928972"] # the article does not have an author and is not in proc_auths_exp

,url,year,content_type,jid,auth_num,raw,resolution,aff_num,affiliation,id,id_author
